In [ ]:
from jax import grad, jit
import jax
import jax.numpy as np
import matplotlib.pyplot as plt
import spectrum_recovery_pool as srp
import copy
import numpy
from skimage import io
from matplotlib.patches import Rectangle

from lippmann import show_spectrum, show_lippmann_transform, lippmann_transform
from display_spectral_data import load_specim_data

# Alternative optimisation
Using autograd and jax.  
The goal of this notebook is to optimise the spectrum assuming it's smooth, but not necessairly bandlimited, see regularisation. 

I did not estimate depth or decay here.

In [ ]:
name = "parrot" # prefix of the file with data and results 
    
# if the data is stored in "Cubes"     
downsampled, wavelengths = load_specim_data("Cubes/" + name, ds=1, cut=True)

image = io.imread("Cubes/" + name + ".png")
image = np.swapaxes(image[:, ::-1, :3], 1, 0)
cut_idx = numpy.loadtxt("Cubes/" + name + "_cut.txt").astype(int)
image = image[cut_idx[0, 0]:cut_idx[0, 1], cut_idx[1, 0]:cut_idx[1, 1]]

pixel = (160, 100)

plt.figure(figsize=(5, 5))
rect = Rectangle((pixel[0] - 2, pixel[1] - 2), 4, 4, alpha=1, color="none", ec="white", lw=1,zorder=10)
plt.gca().add_patch(rect)
plt.imshow(image)
plt.show()

In [ ]:
Z = 10e-6
r_mercury = 0.7 * np.exp(1j * np.deg2rad(148))
r_air = 0.2
r = r_mercury
k0 = 3.7

omegas = 2 * np.pi * srp.c / wavelengths

def forward_model(power_spectrum):
    A = srp.generate_matrix_A(omegas, Z, r=r, k0=k0)
    return np.abs(A @ power_spectrum)**2

measured_spectrum = downsampled[pixel]

def data_fidelity(power_spectrum):
    return np.linalg.norm(measured_spectrum - forward_model(power_spectrum))

### Regularisation
Just means square penalty on the first derivative seems to be working well, but we could use second derivative too

In [ ]:
def derivative_regularisation(power_spectrum):
    derivative = np.linalg.norm(jax.device_put(power_spectrum)[1:]-jax.device_put(power_spectrum)[:-1])
    return derivative

def second_derivative_regularisation(power_spectrum):
    derivative = np.linalg.norm(jax.device_put(power_spectrum)[2:]-2*jax.device_put(power_spectrum)[1:-1] + jax.device_put(power_spectrum)[:-2])
    return derivative

### Initialise with the recorded spectrum

In [ ]:
power_spectrum = copy.copy(measured_spectrum)

In [ ]:
show_spectrum(wavelengths, measured_spectrum, show_background=True)
plt.show()

In [ ]:
show_spectrum(wavelengths, power_spectrum, show_background=True)
plt.show()

In [ ]:
data_grad = grad(data_fidelity)
regularization_grad = grad(regularization)
plt.plot(wavelengths, data_grad(power_spectrum))
plt.show()

In [ ]:
plt.plot(wavelengths, regularization_grad(power_spectrum))
plt.show()

In [ ]:
rate = 0.2
cost = 100
for _ in range(1000):
    power_spectrum -= rate * (data_grad(power_spectrum) + 0.1 * regularization_grad(power_spectrum))
    power_spectrum = np.maximum(0, power_spectrum) # Force power_spectrum to be non-negative
    if cost < data_fidelity(power_spectrum):
        rate = 0.5 * rate
        print("lowering the rate")
    cost = data_fidelity(power_spectrum)
    print(cost)


In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(16, 4))

show_spectrum(wavelengths, measured_spectrum, show_background=True, ax=ax1)
show_spectrum(wavelengths, power_spectrum, show_background=True, ax=ax2)

depths = np.linspace(0,Z,200)
show_lippmann_transform(depths, lippmann_transform(wavelengths, measured_spectrum, depths, r=r, k0=k0)[0], ax=ax3, short_display=True)

show_spectrum(wavelengths, forward_model(power_spectrum), show_background=True, ax=ax4)
ax1.set_title("Reflected spectrum")
ax2.set_title("Curent estimate")
ax3.set_title("Pattern")
ax4.set_title("Re-estimated reflected spectrum")
plt.show()

In [ ]:
plt.figure(figsize=(5,5))
plt.plot(wavelengths, measured_spectrum)
plt.plot(wavelengths, forward_model(power_spectrum))
plt.show()